In [ ]:

[root@bosprod01 Downloads]# ifconfig -a

inet 192.168.153.139  netmask 255.255.255.0  broadcast 192.168.153.255

[root@bosprod02 Downloads]# ifconfig -a

inet 192.168.153.140  netmask 255.255.255.0  broadcast 192.168.153.255

[root@bosprod03 ~]# ifconfig -a

inet 192.168.153.141  netmask 255.255.255.0  broadcast 192.168.153.255



sed -i 's/mirrorlist/#mirrorlist/g' /etc/yum.repos.d/CentOS-*

sed -i 's|#baseurl=http://mirror.centos.org|baseurl=http://vault.centos.org|g' /etc/yum.repos.d/CentOS-*

yum localinstall mysql84-community-release-el8-1.noarch.rpm 

dnf config-manager --disable mysql-8.4-lts-community

dnf config-manager --enable  mysql80-community

yum module disable mysql

yum install mysql-community-server-*8.0.30* --skip-broken

systemctl start mysqld

systemctl status mysqld

systemctl enable mysqld.service

grep 'temporary password' /var/log/mysqld.log

mysql_secure_installation

In [ ]:

Start 2 servers bosprod01 (primary) & bosprod02 (replica) with GTID enabled 
=================================================================================

[root@bosprod01 ~]# cat /etc/my.cnf
[mysqld]

datadir=/var/lib/mysql
socket=/var/lib/mysql/mysql.sock

log-error=/var/log/mysqld.log
pid-file=/var/run/mysqld/mysqld.pid

log-bin = /var/lib/mysql/binlog/bosprod01_binlog
log-bin-index = /var/lib/mysql/binlog/bosprod01_binlog.index
binlog-format = MIXED

gtid-mode = ON
enforce-gtid-consistency = ON

server-id = 1

[root@bosprod01 ~]# mkdir -p /var/lib/mysql/binlog
[root@bosprod01 ~]# 
[root@bosprod01 ~]# chown -R mysql:mysql /var/lib/mysql/binlog
[root@bosprod01 ~]# 


[root@bosprod01 ~]# systemctl stop mysqld
[root@bosprod01 ~]# 
[root@bosprod01 ~]# systemctl start mysqld
[root@bosprod01 ~]# 



[root@bosprod02 ~]# cat /etc/my.cnf
[mysqld]

datadir=/var/lib/mysql
socket=/var/lib/mysql/mysql.sock

log-error=/var/log/mysqld.log
pid-file=/var/run/mysqld/mysqld.pid

log-bin = /var/lib/mysql/binlog/bosprod02_binlog
log-bin-index = /var/lib/mysql/binlog/bosprod02_binlog.index
relay-log = /var/lib/mysql/relaylog/bosprod02_relaylog
relay-log-index = /var/lib/mysql/relaylog/bosprod02_relaylog.index

binlog-format = MIXED

skip-replica-start

gtid-mode = ON
enforce-gtid-consistency = ON

server-id = 2

[root@bosprod02 ~]# mkdir -p /var/lib/mysql/binlog
[root@bosprod02 ~]# mkdir -p /var/lib/mysql/relaylog
[root@bosprod02 ~]# 
[root@bosprod02 ~]# chown -R mysql:mysql /var/lib/mysql/binlog
[root@bosprod02 ~]# chown -R mysql:mysql /var/lib/mysql/relaylog
[root@bosprod02 ~]# 


[root@bosprod02 ~]# systemctl stop mysqld
[root@bosprod02 ~]# 
[root@bosprod02 ~]# systemctl start mysqld
[root@bosprod02 ~]#


In [ ]:
Create replication user on both the servers ( when both servers are newly built )
============================================================================================

mysql> create user replicator identified by 'Repl@2024';
Query OK, 0 rows affected (0.03 sec)

mysql> grant replication slave on *.* to replicator;
Query OK, 0 rows affected (0.01 sec)


In [ ]:
Add host ips in /etc/hosts file and stop firewalld in both servers 
=======================================================

[root@bosprod02 ~]# cat /etc/hosts
127.0.0.1   localhost localhost.localdomain localhost4 localhost4.localdomain4
::1         localhost localhost.localdomain localhost6 localhost6.localdomain6

192.168.153.139 bosprod01
192.168.153.140 bosprod02
192.168.153.141 bosprod03
[root@bosprod02 ~]# 

[root@bosprod02 ~]# ping bosprod01
PING bosprod01 (192.168.153.139) 56(84) bytes of data.
64 bytes from bosprod01 (192.168.153.139): icmp_seq=1 ttl=64 time=0.773 ms
64 bytes from bosprod01 (192.168.153.139): icmp_seq=2 ttl=64 time=1.03 ms
64 bytes from bosprod01 (192.168.153.139): icmp_seq=3 ttl=64 time=0.689 ms

[root@bosprod01 ~]# ping bosprod02
PING bosprod02 (192.168.153.140) 56(84) bytes of data.
64 bytes from bosprod02 (192.168.153.140): icmp_seq=1 ttl=64 time=0.646 ms
64 bytes from bosprod02 (192.168.153.140): icmp_seq=2 ttl=64 time=1.01 ms
64 bytes from bosprod02 (192.168.153.140): icmp_seq=3 ttl=64 time=0.897 ms


[root@bosprod01 ~]# systemctl stop firewalld
[root@bosprod01 ~]# 

[root@bosprod02 ~]# systemctl stop firewalld
[root@bosprod02 ~]# 


In [ ]:
Configure the replica to use GTID-based auto positioning 
======================================================================

mysql> change replication source to
    -> source_host = 'bosprod01',
    -> source_port = 3306,
    -> source_user = 'replicator',
    -> source_password = 'Repl@2024',
    -> source_auto_position = 1;
Query OK, 0 rows affected, 2 warnings (0.04 sec)

mysql> start replica;
Query OK, 0 rows affected (0.05 sec)

mysql> show replica status \G
*************************** 1. row ***************************
             Replica_IO_State: Connecting to source
                  Source_Host: bosprod01


2024-06-11T19:32:54.304599Z 12 [ERROR] [MY-010584] [Repl] Slave SQL for channel '': Worker 1 failed executing transaction 
'c9557c55-27c4-11ef-89ce-000c29051378:1' at master log bosprod01_binlog.000001, end_log_pos 476; 
Error 'Operation CREATE USER failed for 'replicator'@'%'' on query. Default database: ''. 
Query: 'CREATE USER 'replicator'@'%' IDENTIFIED WITH 'caching_sha2_password' AS '$A$005$qa4`y`(,x\rC
                                               XG42JpVhFs90q7iBrpjqnMOt5VoXEHnWQ4WjCaVcT4r7'', Error_code: MY-001396
2024-06-11T19:32:54.335999Z 11 [ERROR] [MY-010586] [Repl] Error running query, slave SQL thread aborted. Fix the problem, 
and restart the slave SQL thread with "SLAVE START". We stopped at log 'bosprod01_binlog.000001' position 157


mysql> stop replica;
Query OK, 0 rows affected (0.00 sec)

mysql> drop user replicator;
Query OK, 0 rows affected (0.00 sec)

mysql> start replica;
Query OK, 0 rows affected (0.29 sec)

mysql> show replica status \G
*************************** 1. row ***************************
             Replica_IO_State: Waiting for source to send event
                  Source_Host: bosprod01
                  Source_User: replicator
                  Source_Port: 3306
                Connect_Retry: 60
              Source_Log_File: bosprod01_binlog.000001
          Read_Source_Log_Pos: 702
               Relay_Log_File: bosprod02_relaylog.000003
                Relay_Log_Pos: 481
        Relay_Source_Log_File: bosprod01_binlog.000001
           Replica_IO_Running: Yes
          Replica_SQL_Running: Yes

        Seconds_Behind_Source: 0

             Source_Server_Id: 1
                  Source_UUID: c9557c55-27c4-11ef-89ce-000c29051378

           Retrieved_Gtid_Set: c9557c55-27c4-11ef-89ce-000c29051378:1-2
            Executed_Gtid_Set: b238670c-27c9-11ef-8825-000c293ea0bc:1-3,
                               c9557c55-27c4-11ef-89ce-000c29051378:1-2

In [ ]:
Create a database in primary and insert some data 
================================================================

mysql> create database bosp;
mysql> use bosp;
mysql> create table orders (order_id int primary key , order_details varchar(30) , order_date datetime );
mysql> call sp_insert_orders (1);
mysql> select count(*) from orders;
+----------+
| count(*) |
+----------+
|   100000 |
+----------+
1 row in set (0.14 sec)


mysql> select @@hostname;
+------------+
| @@hostname |
+------------+
| bosprod02  |
+------------+
1 row in set (0.00 sec)

mysql> select count(*) from orders;
+----------+
| count(*) |
+----------+
|   100000 |
+----------+
1 row in set (0.13 sec)


In [ ]:
Add another replica ( bosprod03 ) to the replication environment 
============================================================================


remove data directory files from new replica server ( bosprod03 )
--------------------------------------------------------------------

[root@bosprod03 ~]# systemctl stop mysqld
[root@bosprod03 ~]# 

[root@bosprod03 ~]# rm -rf /var/lib/mysql/*
[root@bosprod03 ~]# 


stop the existing replica and copy the data directory including log files and relay log files
into the new replica server.
------------------------------------------------------------------------------------------------------

mysql> stop replica;
Query OK, 0 rows affected (0.32 sec)

[root@bosprod02 ~]# systemctl stop mysqld
[root@bosprod02 ~]# 


[root@bosprod02 ~]# scp -r /var/lib/mysql/* root@bosprod03:/var/lib/mysql/
root@bosprod03's password: 
auto.cnf                          100%   56    30.6KB/s   00:00    
bosprod02_binlog.000001           100%   39MB  14.3MB/s   00:02    
bosprod02_binlog.index            100%   46    22.1KB/s   00:00    


after copying , delete the auto.cnf in new replica server ( bosprod03 ) so that a new 
server uuid is created in the new server.
-------------------------------------------------------------------------------------------

[root@bosprod03 ~]# chown -R mysql:mysql /var/lib/mysql
[root@bosprod03 ~]# 

[root@bosprod03 ~]# rm -rf /var/lib/mysql/auto.cnf 
[root@bosprod03 ~]# 


edit the config file in new replica server to add replication options 
------------------------------------------------------------------------------

[root@bosprod03 ~]# cat /etc/my.cnf
[mysqld]

datadir=/var/lib/mysql
socket=/var/lib/mysql/mysql.sock

log-error=/var/log/mysqld.log
pid-file=/var/run/mysqld/mysqld.pid

log-bin = /var/lib/mysql/binlog/bosprod02_binlog
log-bin-index = /var/lib/mysql/binlog/bosprod02_binlog.index
relay-log = /var/lib/mysql/relaylog/bosprod02_relaylog
relay-log-index = /var/lib/mysql/relaylog/bosprod02_relaylog.index

binlog-format = MIXED

skip-replica-start

gtid-mode = ON
enforce-gtid-consistency = ON

server-id = 3


restart the old replica ( bosprod02 )
===========================================================

[root@bosprod02 ~]# systemctl start mysqld
[root@bosprod02 ~]# 

mysql> start replica;
Query OK, 0 rows affected (0.26 sec)

mysql> show slave status \G


start the new replica ( bosprod03 )
=============================================================

[root@bosprod03 ~]# systemctl start mysqld
Job for mysqld.service failed because the control process exited with error code.
See "systemctl status mysqld.service" and "journalctl -xe" for details.
[root@bosprod03 ~]# 


[root@bosprod03 ~]# tail -100 /var/log/mysqld.log 

2024-06-11T20:58:05.863872Z 0 [System] [MY-010116] [Server] /usr/sbin/mysqld (mysqld 8.0.30) 
starting as process 3890
mysqld: File '/var/lib/mysql/binlog/bosprod02_binlog.index' not found (OS errno 13 - Permission denied)
2024-06-11T20:58:05.868520Z 0 [ERROR] [MY-010119] [Server] Aborting
2024-06-11T20:58:05.868857Z 0 [System] [MY-010910] [Server] /usr/sbin/mysqld: Shutdown complete (mysqld 8.0.30)  M
ySQL Community Server - GPL.

[root@bosprod03 ~]# 

[root@bosprod03 ~]# ls -rtl /var/lib/mysql/binlog/
total 40416
-rw-r-----. 1 mysql mysql 41380972 Jun 12 02:10 bosprod02_binlog.000001
-rw-r-----. 1 mysql mysql       46 Jun 12 02:10 bosprod02_binlog.index
[root@bosprod03 ~]# 


[root@bosprod03 var]# restorecon -Rv /var/lib/mysql

Relabeled /var/lib/mysql/binlog from unconfined_u:ob
Relabeled /var/lib/mysql/undo_001 from unconfined_u:object_r:var_lib_t:s0 to unconfined_u:object_r:mysqld_db_t:s0
Relabeled /var/lib/mysql/undo_002 from unconfined_u:object_r:var_lib_t:s0 to unconfined_u:object_r:mysqld_db_t:s0


[root@bosprod03 var]# systemctl start mysqld
[root@bosprod03 var]# 


mysql> start replica;
Query OK, 0 rows affected (0.03 sec)

mysql> show slave status \G
*************************** 1. row ***************************
               Slave_IO_State: Queueing source event to the relay log
                  Master_Host: bosprod01
                  Master_User: replicator
                  Master_Port: 3306
                Connect_Retry: 60
              Master_Log_File: bosprod01_binlog.000001
          Read_Master_Log_Pos: 60503964
               Relay_Log_File: bosprod02_relaylog.000005
                Relay_Log_Pos: 237292
        Relay_Master_Log_File: bosprod01_binlog.000001
             Slave_IO_Running: Yes
            Slave_SQL_Running: Yes

            Seconds_Behind_Master: 2103

            

mysql> select @@hostname , count(*) from orders;
+------------+----------+
| @@hostname | count(*) |
+------------+----------+
| bosprod01  |   400000 |
+------------+----------+
1 row in set (0.29 sec)


mysql> select @@hostname , count(*) from orders;
+------------+----------+
| @@hostname | count(*) |
+------------+----------+
| bosprod02  |   400000 |
+------------+----------+
1 row in set (0.37 sec)

mysql> select @@hostname , count(*) from orders;
+------------+----------+
| @@hostname | count(*) |
+------------+----------+
| bosprod03  |   188042 |
+------------+----------+
1 row in set (0.02 sec)


mysql> show replicas;
+-----------+------+------+-----------+--------------------------------------+
| Server_Id | Host | Port | Source_Id | Replica_UUID                         |
+-----------+------+------+-----------+--------------------------------------+
|         3 |      | 3306 |         1 | bcd7bc13-2837-11ef-92fe-000c29009988 |
|         2 |      | 3306 |         1 | b238670c-27c9-11ef-8825-000c293ea0bc |
+-----------+------+------+-----------+--------------------------------------+
2 rows in set (0.00 sec)


mysql> select @@hostname , count(*) from orders;
+------------+----------+
| @@hostname | count(*) |
+------------+----------+
| bosprod03  |   400000 |
+------------+----------+
1 row in set (0.03 sec)


In [ ]:
Using replication for backups
================================================


For consistent backup we can shutdown replica and take mysqldump backup , alternatively we can use stop 
replica sql_thread and take the backup from replica
----------------------------------------------------------------------------------------------------------------


mysql> select @@hostname;
| bosprod02  |

mysql> stop replica sql_thread;

mysql> show replica status \G
*************************** 1. row ***************************
           Replica_IO_Running: Yes
          Replica_SQL_Running: No

[root@bosprod02 ~]# mysqldump -u root -p --all-databases --routines > /home/dbaadmin/backups/fulldb_backup.sql

mysql> start replica sql_thread;

mysql> show replica status \G
*************************** 1. row ***************************
           Replica_IO_Running: Yes
          Replica_SQL_Running: Yes


In [ ]:
Views and tables to check replication
==================================================================


in primary 
------------------------

mysql> select * from mysql.gtid_executed;
+--------------------------------------+----------------+--------------+
| source_uuid                          | interval_start | interval_end |
+--------------------------------------+----------------+--------------+
| c9557c55-27c4-11ef-89ce-000c29051378 |              1 |       497839 |
| c9557c55-27c4-11ef-89ce-000c29051378 |         497840 |       497888 |

| c9557c55-27c4-11ef-89ce-000c29051378 |         499923 |       499972 |
| c9557c55-27c4-11ef-89ce-000c29051378 |         499973 |       500007 |
+--------------------------------------+----------------+--------------+
45 rows in set (0.00 sec)


mysql> show master status;
+-------------------------+----------+--------------+------------------+-----------------------------------------------+
| File                    | Position | Binlog_Do_DB | Binlog_Ignore_DB | Executed_Gtid_Set                             |
+-------------------------+----------+--------------+------------------+-----------------------------------------------+
| bosprod01_binlog.000002 | 40900197 |              |                  | c9557c55-27c4-11ef-89ce-000c29051378:1-500007 |
+-------------------------+----------+--------------+------------------+-----------------------------------------------+
1 row in set (0.00 sec)


mysql> show replicas;
+-----------+------+------+-----------+--------------------------------------+
| Server_Id | Host | Port | Source_Id | Replica_UUID                         |
+-----------+------+------+-----------+--------------------------------------+
|         3 |      | 3306 |         1 | bcd7bc13-2837-11ef-92fe-000c29009988 |
|         2 |      | 3306 |         1 | b238670c-27c9-11ef-8825-000c293ea0bc |
+-----------+------+------+-----------+--------------------------------------+
2 rows in set (0.00 sec)


mysql> select * from performance_schema.processlist  where user = 'replicator';
+----+------------+-----------------+------+------------------+------+-------------------------------------------------
| ID | USER       | HOST            | DB   | COMMAND          | TIME | STATE                                             
+----+------------+-----------------+------+------------------+------+--------------------------------------------------
| 30 | replicator | bosprod03:49358 | NULL | Binlog Dump GTID | 7748 | Source has sent all binlog to replica; waiting 
                                                                       for more updates | NULL | PRIMARY          |
| 32 | replicator | bosprod02:41386 | NULL | Binlog Dump GTID | 1537 | Source has sent all binlog to replica; waiting 
                                                                       for more updates | NULL | PRIMARY          |
+----+------------+-----------------+------+------------------+------+--------------------------------------------------
2 rows in set (0.00 sec)


in replica 
---------------------------------------------------------

mysql> show replica status \G
*************************** 1. row ***************************
             Replica_IO_State: Waiting for source to send event
                  Source_Host: bosprod01
                  Source_User: replicator
                  Source_Port: 3306
                Connect_Retry: 60
              Source_Log_File: bosprod01_binlog.000002
          Read_Source_Log_Pos: 40900197
               Relay_Log_File: bosprod02_relaylog.000007
                Relay_Log_Pos: 40900427
        Relay_Source_Log_File: bosprod01_binlog.000002
           Replica_IO_Running: Yes
          Replica_SQL_Running: Yes

mysql> select * from mysql.slave_master_info;


mysql> select * from performance_schema.replication_connection_status \G
*************************** 1. row ***************************
                                      CHANNEL_NAME: 
                                        GROUP_NAME: 
                                       SOURCE_UUID: c9557c55-27c4-11ef-89ce-000c29051378
                                         THREAD_ID: 51
                                     SERVICE_STATE: ON
                         COUNT_RECEIVED_HEARTBEATS: 173

mysql> select * from performance_schema.replication_connection_configuration \G

mysql> select * from performance_schema.replication_applier_status;
+--------------+---------------+-----------------+----------------------------+
| CHANNEL_NAME | SERVICE_STATE | REMAINING_DELAY | COUNT_TRANSACTIONS_RETRIES |
+--------------+---------------+-----------------+----------------------------+
|              | ON            |            NULL |                          0 |
+--------------+---------------+-----------------+----------------------------+
1 row in set (0.00 sec)

mysql> select channel_name,thread_id,service_state 
    -> from performance_schema.replication_applier_status_by_coordinator;
+--------------+-----------+---------------+
| channel_name | thread_id | service_state |
+--------------+-----------+---------------+
|              |        67 | ON            |
+--------------+-----------+---------------+
1 row in set (0.00 sec)


mysql> select channel_name,worker_id,thread_id,service_state from 
performance_schema.replication_applier_status_by_worker;

+--------------+-----------+-----------+---------------+
| channel_name | worker_id | thread_id | service_state |
+--------------+-----------+-----------+---------------+
|              |         1 |        68 | ON            |
|              |         2 |        69 | ON            |
|              |         3 |        70 | ON            |
|              |         4 |        71 | ON            |
+--------------+-----------+-----------+---------------+
4 rows in set (0.01 sec)


In [ ]:
Make the replicas read only
=================================================


mysql> stop replica;
[root@bosprod02 ~]# systemctl stop mysqld
[root@bosprod02 ~]# 

[root@bosprod02 ~]# cat /etc/my.cnf
read-only = TRUE

mysql> show variables like '%read_only%';
+-----------------------+-------+
| Variable_name         | Value |
+-----------------------+-------+
| innodb_read_only      | OFF   |
| read_only             | ON    |
| super_read_only       | OFF   |
| transaction_read_only | OFF   |
+-----------------------+-------+
4 rows in set (0.00 sec)

mysql> start replica;



mysql> call sp_insert_orders (500001);

mysql> select @@hostname , count(*) from orders;
| bosprod01  |   600000 |

mysql> select @@hostname , count(*) from orders;
| bosprod02  |   600000 |

mysql> select @@hostname , count(*) from orders;
| bosprod03  |   600000 |


In [ ]:
Switching Sources during failover 
=================================================

suppose the soure/primary server ( bosprod01 ) is down. Make bosprod02 primary and link replica bosprod03 to 
new primary bosprod02.


[root@bosprod01 ~]# systemctl stop mysqld
[root@bosprod01 ~]# 


[root@bosprod02 ~]# 

mysql> select @@hostname;
| bosprod02  |

mysql> stop replica;

mysql> reset slave all;

mysql> show replica status \G
Empty set (0.00 sec)

remove read-only , relay-log and skip-slave-start options in the config file.

[root@bosprod02 ~]# vi /etc/my.cnf
[root@bosprod02 ~]# 

[root@bosprod02 ~]# systemctl restart mysqld
[root@bosprod02 ~]# 


mysql> select @@hostname;
| bosprod03  |

mysql> stop replica;

mysql> change replication source to
    -> source_host = 'bosprod02',
    -> source_port = 3306,
    -> source_user = 'replicator',
    -> source_password = 'Repl@2024',
    -> source_auto_position = 1;

mysql> start replica;
Query OK, 0 rows affected (0.02 sec)

mysql> show replica status \G
*************************** 1. row ***************************
             Replica_IO_State: Waiting for source to send event
                  Source_Host: bosprod02
                  Source_User: replicator
                  Source_Port: 3306
                Connect_Retry: 60
              Source_Log_File: bosprod02_binlog.000005
          Read_Source_Log_Pos: 237
               Relay_Log_File: bosprod02_relaylog.000004
                Relay_Log_Pos: 427
        Relay_Source_Log_File: bosprod02_binlog.000005
           Replica_IO_Running: Yes
          Replica_SQL_Running: Yes


mysql> select @@hostname;
| bosprod02  |

mysql> show replicas;
+-----------+------+------+-----------+--------------------------------------+
| Server_Id | Host | Port | Source_Id | Replica_UUID                         |
+-----------+------+------+-----------+--------------------------------------+
|         3 |      | 3306 |         2 | bcd7bc13-2837-11ef-92fe-000c29009988 |
+-----------+------+------+-----------+--------------------------------------+

mysql> call sp_insert_orders (600001);

mysql> select @@hostname , count(*) from orders;
| bosprod02  |   700000 |

mysql> select @@hostname , count(*) from orders;
| bosprod03  |   700000 |


Once the old primary is up , we can make it a replica of new primary 
------------------------------------------------------------------------------

mysql> change replication source to
    -> source_host = 'bosprod02',
    -> source_port = 3306,
    -> source_user = 'replicator',
    -> source_password = 'Repl@2024',
    -> source_auto_position = 1;
Query OK, 0 rows affected, 2 warnings (0.08 sec)

mysql> start replica;
Query OK, 0 rows affected (0.20 sec)

mysql> show replica status \G
                  Last_Errno: 1396
                  Last_Error: Coordinator stopped because there were error(s) in the worker(s). The most recent failure being:
                   Worker 1 failed executing transaction 'b238670c-27c9-11ef-8825-000c293ea0bc:1' at master log bosprod02_binlog.000001, 
                   end_log_pos 477. See error log and/or performance_schema.replication_applier_status_by_worker table for more
                   details about this failure or others, if any.

mysql> select * from performance_schema.replication_applier_status_by_worker;

       Worker 1 failed executing transaction 'b238670c-27c9-11ef-8825-000c293ea0bc:1' at master log bosprod02_binlog.000001, 
       end_log_pos 477; Error 'Operation CREATE USER failed for 'replicator'@'%'' on query. Default database: ''. 
       Query: 'CREATE USER 'replicator'@'%' IDENTIFIED WITH 'caching_sha2_password' AS '$A$005$bcJPk{R<\Z\"`

mysql> drop user replicator;

mysql> stop replica;

mysql> start replica;

mysql> show replica status \G
*************************** 1. row ***************************
             Replica_IO_State: waiting for handler commit
             Replica_IO_State: Waiting for source to send event
                  Source_Host: bosprod02
                  Source_User: replicator
                  Source_Port: 3306
        
           Replica_IO_Running: Yes
          Replica_SQL_Running: Yes
    
        Seconds_Behind_Source: 919
                Last_IO_Errno: 0
                Last_IO_Error: 
               Last_SQL_Errno: 0
               Last_SQL_Error: 

mysql> show replicas;
+-----------+------+------+-----------+--------------------------------------+
| Server_Id | Host | Port | Source_Id | Replica_UUID                         |
+-----------+------+------+-----------+--------------------------------------+
|         1 |      | 3306 |         2 | c9557c55-27c4-11ef-89ce-000c29051378 |
|         3 |      | 3306 |         2 | bcd7bc13-2837-11ef-92fe-000c29009988 |
+-----------+------+------+-----------+--------------------------------------+
2 rows in set (0.00 sec)

mysql> select @@hostname , count(*) from orders;
| bosprod02  |   800000 |
mysql> select @@hostname , count(*) from orders;
| bosprod01  |   800000 |
mysql> select @@hostname , count(*) from orders;
| bosprod03  |   800000 |